In [1]:
import dgl
import dgl.function as fn
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph
import numpy as np
import pickle
import networkx as nx
from dataloader import *
from RGCN import *
from model import *

Using backend: pytorch


In [2]:
with open("DGL_graph.pkl", "rb") as f:
    g = pickle.load(f)

with open("data/conceptnet/embedding_values.pkl", "rb") as f:
    embedding_values = pickle.load(f)
print("-------------- Finish loading -------------")
g.ndata["x"] = embedding_values
# subsample a strongly-connected subgraph

G_nx = g.to_networkx()
sub_G_nx = nx.strongly_connected_components(G_nx)
SCC = []
for item in sub_G_nx:
    if len(item) > 2:
        SCC.append(item)
component = list(SCC[0])
print("-------------- Training data subsampled -------------")
# assign embedding to graph
sub_graph = g.subgraph(component)
sub_graph.copy_from_parent()
sub_graph_G = sub_graph

#X,y = gen_training_set(sub_graph_G, 3, 1000)

with open("training_data.pkl", "rb") as f:
    dic = pickle.load(f)
X,y = dic["X"], dic["y"]
print("-------------- Training data generated -------------")

-------------- Finish loading -------------
-------------- Training data subsampled -------------
-------------- Training data generated -------------


In [3]:
from transformers import BertTokenizer
tokenizer = BertTokenizer("data/conceptnet/embedding_keys.txt")

In [7]:
with open("data/conceptnet/embedding_keys.pkl", "rb") as f:
    embedding_keys = pickle.load(f)
for i,item in enumerate(embedding_keys):
    words = item.replace("_", " ")
    embedding_keys[i] = words
embedding_keys = np.array(embedding_keys)
index2phrase = embedding_keys[component]

In [8]:
CORPUS_SIZE = len(tokenizer.vocab)
input_size = 300
hidden_size = 128
node_output_size = 2
phrase_output_size = CORPUS_SIZE
num_rels = 34
batch_size = 1
num_batch = 2
n_hidden_layers = 2
n_bases = -1
device = torch.device("cuda")

node_generator = LSTM_node_generator(hidden_size, node_output_size, batch_size)
graph_encoder = Model(input_size,
            hidden_size,
            hidden_size,
            num_rels,
            num_bases=n_bases,
            num_hidden_layers=n_hidden_layers).to(device)
phrase_generator = LSTM_phrase_generator(hidden_size, phrase_output_size)

In [10]:
device = torch.device("cuda")
graph_encoder.to(device)
node_generator.to(device)
phrase_generator.to(device)

CLS_token = 0
SEP_token = 1

In [11]:
g_list = []
y_list = []
for i in range(100):
    g = sub_graph_G.subgraph(X[i])
    g.copy_from_parent()
    g.ndata["x"] = g.ndata["x"].float().to(device)
    edge_norm = torch.ones(g.edata['rel_type'].shape[0]).to(device)
    g.edata.update({'norm': edge_norm.view(-1,1).to(device)})
    g_list.append(g)
    y_list.append(y[i])

In [18]:
def gen_phrase(i, phrase_decoder, new_node_embedding, target):
    loss = 0
    index = target[i]
    y_tokenize = torch.Tensor(tokenizer(index2phrase[index])['input_ids'][1:]).view(batch_size,-1).long().to(device)
    output = []
    phrase_decoder_input = torch.tensor([CLS_token], device=device).view(1, 1)
    phrase_decoder_hidden = new_node_embedding
    for ni in range(y_tokenize.shape[1]):
        phrase_decoder_output, node_decoder_hidden = phrase_generator(
            phrase_decoder_input, phrase_decoder_hidden)
        topv, topi = phrase_decoder_output.topk(1)
        if teacher_forcing:
            phrase_decoder_input = y_tokenize[:,ni].view(batch_size, 1)
        else:
            phrase_decoder_input = topi.squeeze().detach()  # detach from history as input 

        loss += criterion(phrase_decoder_output.view(batch_size,-1), y_tokenize[:, ni].view(batch_size))
        output.append(topi.squeeze().detach().cpu().numpy())
    #loss.backward(retain_graph = True)
    #node_decoder_optimizer.step()
    #graph_encoder_optimizer.step()
    #phrase_decoder_optimizer.step()
    return loss, tokenizer.decode(output), y_tokenize

In [25]:
#CUDA_LAUNCH_BLOCKING=1
learning_rate = 1e-3
teacher_forcing = True
target = torch.Tensor([[0,0,1]]).to(device).long()

#graph_encoder_optimizer = optim.Adam(graph_encoder.parameters(), lr=learning_rate)
#node_decoder_optimizer = optim.Adam(node_generator.parameters(), lr=learning_rate)
#phrase_decoder_optimizer = optim.Adam(phrase_generator.parameters(), lr=learning_rate)

for i in range(200):
    for j in range(200):
        node_decoder_optimizer.zero_grad()
        graph_encoder_optimizer.zero_grad()
        phrase_decoder_optimizer.zero_grad()

        node_embedding, g_embedding = graph_encoder(g_list[j])
        node_decoder_input = torch.tensor([[CLS_token] * batch_size], device=device).view(batch_size, 1)
        node_decoder_hidden = (g_embedding.view(1,batch_size,-1), g_embedding.view(1,batch_size,-1))

        output = []
        phrase_output = []
        criterion = nn.CrossEntropyLoss()
        loss = 0
        # Generate new nodes
        for ni in range(target.shape[1]):
            node_decoder_output, node_decoder_hidden = node_generator(
                node_decoder_input, node_decoder_hidden)
            new_node_embedding = node_decoder_hidden
            topv, topi = node_decoder_output.topk(1)
            if teacher_forcing:
                node_decoder_input = target[:,ni].view(batch_size, 1)
            else:
                node_decoder_input = topi.squeeze().detach()  # detach from history as input 
            output.append(topi.squeeze().detach().cpu().numpy())
            loss += criterion(node_decoder_output.view(batch_size,-1), target[:, ni].view(batch_size))
            phrase_loss, p_output, _ = gen_phrase(ni, phrase_generator, new_node_embedding, y[j])
            phrase_output.append(p_output)
            loss += phrase_loss

        loss.backward(retain_graph = True)
        node_decoder_optimizer.step()
        graph_encoder_optimizer.step()
        phrase_decoder_optimizer.step()
    if i % 10 == 0:
        print(loss, phrase_output)

tensor(27.5050, device='cuda:0', grad_fn=<AddBackward0>) ['purse [SEP]', 'backpack [SEP]', 'drive [SEP]']
tensor(3.5912, device='cuda:0', grad_fn=<AddBackward0>) ['scientist [SEP]', 'baby [SEP]', 'tooth [SEP]']
tensor(2.0513, device='cuda:0', grad_fn=<AddBackward0>) ['scientist [SEP]', 'baby [SEP]', 'tooth [SEP]']
tensor(0.6603, device='cuda:0', grad_fn=<AddBackward0>) ['scientist [SEP]', 'baby [SEP]', 'tooth [SEP]']
tensor(0.2918, device='cuda:0', grad_fn=<AddBackward0>) ['scientist [SEP]', 'baby [SEP]', 'tooth [SEP]']
tensor(0.1276, device='cuda:0', grad_fn=<AddBackward0>) ['scientist [SEP]', 'baby [SEP]', 'tooth [SEP]']
tensor(0.1636, device='cuda:0', grad_fn=<AddBackward0>) ['scientist [SEP]', 'baby [SEP]', 'tooth [SEP]']
tensor(0.0973, device='cuda:0', grad_fn=<AddBackward0>) ['scientist [SEP]', 'baby [SEP]', 'tooth [SEP]']
tensor(0.3306, device='cuda:0', grad_fn=<AddBackward0>) ['scientist [SEP]', 'baby [SEP]', 'tooth [SEP]']
tensor(0.0265, device='cuda:0', grad_fn=<AddBackward0>

In [26]:
g_list = []
y_list = []
for i in range(200):
    g = sub_graph_G.subgraph(X[i])
    g.copy_from_parent()
    g.ndata["x"] = g.ndata["x"].float().to(device)
    edge_norm = torch.ones(g.edata['rel_type'].shape[0]).to(device)
    g.edata.update({'norm': edge_norm.view(-1,1).to(device)})
    g_list.append(g)
    y_list.append(y[i])

for j in range(100,200):
    node_embedding, g_embedding = graph_encoder(g_list[j])
    node_decoder_input = torch.tensor([[CLS_token] * batch_size], device=device).view(batch_size, 1)
    node_decoder_hidden = (g_embedding.view(1,batch_size,-1), g_embedding.view(1,batch_size,-1))

    output = []
    phrase_output = []
    y_token = []
    criterion = nn.CrossEntropyLoss()
    # Generate new nodes
    for ni in range(target.shape[1]):
        node_decoder_output, node_decoder_hidden = node_generator(
            node_decoder_input, node_decoder_hidden)
        new_node_embedding = node_decoder_hidden
        topv, topi = node_decoder_output.topk(1)
        if teacher_forcing:
            node_decoder_input = target[:,ni].view(batch_size, 1)
        else:
            node_decoder_input = topi.squeeze().detach()  # detach from history as input 
        output.append(topi.squeeze().detach().cpu().numpy())
        phrase_loss, p_output, y_tokenize = gen_phrase(ni, phrase_generator, new_node_embedding, y[j])
        phrase_output.append(p_output)
        y_token.append(tokenizer.decode(y_tokenize.view(-1)))
    print(phrase_output, y_token)

['mouse [SEP]', 'heart [SEP]', 'entertain [SEP]'] ['fanatic [SEP]', 'heart [SEP]', 'drink [SEP]']
['playroom [SEP]', 'now [SEP]', 'go to war [SEP]'] ['playroom [SEP]', 'wheat [SEP]', 'go to war [SEP]']
['can [SEP]', 'it [SEP]', 'copulate [SEP]'] ['can [SEP]', 'it [SEP]', 'copulate [SEP]']
['pond [SEP]', 'now [SEP]', 'plate [SEP]'] ['pond [SEP]', 'now [SEP]', 'plate [SEP]']
['cook [SEP]', 'exercise [SEP]', 'gambler [SEP]'] ['cook [SEP]', 'exercise [SEP]', 'gambler [SEP]']
['choir [SEP]', 'sadness [SEP]', 'travel [SEP]'] ['choir [SEP]', 'sadness [SEP]', 'travel [SEP]']
['home [SEP]', 'fin [SEP]', 'fish [SEP]'] ['home [SEP]', 'fin [SEP]', 'fish [SEP]']
['sit down [SEP]', 'wash [SEP]', 'this [SEP]'] ['sit down [SEP]', 'wash [SEP]', 'this [SEP]']
['shovel [SEP]', 'woman [SEP]', 'place [SEP]'] ['shovel [SEP]', 'woman [SEP]', 'place [SEP]']
['fan [SEP]', 'this [SEP]', 'orange [SEP]'] ['fan [SEP]', 'this [SEP]', 'orange [SEP]']
['organism [SEP]', 'leaf [SEP]', 'attend rock concert [SEP]'] ['or

['cook food [SEP]', 'erection [SEP]', 'drink [SEP]'] ['cook food [SEP]', 'erection [SEP]', 'drink [SEP]']
['fly [SEP]', 'customer [SEP]', 'visit friend [SEP]'] ['fly [SEP]', 'customer [SEP]', 'visit friend [SEP]']
['this [SEP]', 'stall [SEP]', 'farmhouse [SEP]'] ['this [SEP]', 'stall [SEP]', 'farmhouse [SEP]']
['cry [SEP]', 'storage [SEP]', 'this [SEP]'] ['cry [SEP]', 'storage [SEP]', 'this [SEP]']
['use computer [SEP]', 'bake bread [SEP]', 'cow [SEP]'] ['use computer [SEP]', 'bake bread [SEP]', 'cow [SEP]']
['wool [SEP]', 'library [SEP]', 'alarm clock [SEP]'] ['wool [SEP]', 'library [SEP]', 'alarm clock [SEP]']
['kill person [SEP]', 'muscle [SEP]', 'bullet [SEP]'] ['kill person [SEP]', 'muscle [SEP]', 'bullet [SEP]']
['scientist [SEP]', 'baby [SEP]', 'tooth [SEP]'] ['scientist [SEP]', 'baby [SEP]', 'tooth [SEP]']


### Add edge features

In [6]:
# prepare training set


3388

In [ ]:
index2phrase[0]